## **Preparations**

In [1]:
import os
import sys
import os.path as op
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from functools import partial

import torch

sys.path.append("..")
from mtecg.classifier import ECGClassifier
from mtecg.evaluation import evaluate_from_dataframe_1d
from mtecg.utils import load_ecg_dataframe_1d
import mtecg.constants as constants


SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(SEED)

c:\Anaconda3\envs\ecg\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda3\envs\ecg\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: llow is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
device = "cuda"
round_probabilities = False

singletask_scar_model_path = "../trained_models/1d/single-task-scar/resnet34d_200"
singletask_lvef_model_path = "../trained_models/1d/single-task-lvef/resnet34d_200_LVEF50"
multitask_model_path = "../trained_models/1d/multi-task/resnet34d_200_LVEF50"
multitask_old_format_model_path = "../trained_models/1d/multi-task-old-format/resnet34d_200_LVEF50"
multitask_transferred_model_path = "../trained_models/1d/multi-task-transferred/resnet34d_200_LVEF50"

In [22]:
singletask_scar_classifier = ECGClassifier(
    singletask_scar_model_path,
    model_class="single-task-1d",
    device=device,
    task="scar",
    round_probabilities=round_probabilities,
)
singletask_lvef_classifier = ECGClassifier(
    singletask_lvef_model_path,
    model_class="single-task-1d",
    device=device,
    task="lvef",
    round_probabilities=round_probabilities,
)

multitask_classifier = ECGClassifier(
    multitask_model_path, model_class="multi-task-1d", device=device, round_probabilities=round_probabilities
)
multitask_old_format_classifier = ECGClassifier(
    multitask_old_format_model_path, model_class="multi-task-1d", device=device, round_probabilities=round_probabilities
)
multitask_transferred_classifier = ECGClassifier(
    multitask_transferred_model_path, model_class="multi-task-1d", device=device, round_probabilities=round_probabilities
)

In [3]:
old_csv_path = "../datasets/all_ECG_cleared_duplicate_may23_final_1d_labels_with_rpeaks.csv"
old_data_dir = "../datasets/siriraj_data/ECG_MRI_1d_data_interp"
new_csv_path = "../datasets/all_ECG_cleared_duplicate_may23_final_1d_labels_with_rpeaks.csv"
new_data_dir = "../datasets/siriraj_data/ECG_MRI_1d_data_interp"

In [10]:
# Old test set.
old_test_df = load_ecg_dataframe_1d(
    old_csv_path,
    old_data_dir,
    # imputer_dir=multitask_clinical_model_path,
    do_split=True
)
old_test_df = old_test_df[old_test_df["split"] == "old_test"].reset_index(drop=True)

# New test set. No need to impute.
new_test_df = load_ecg_dataframe_1d(
    new_csv_path,
    new_data_dir,
    # imputer_dir=multitask_clinical_model_path,
    do_split=True,
)

new_test_df = new_test_df[new_test_df["split"] == "new_test"].reset_index(drop=True)

# New test set with lvef_threshold= 40. No need to impute.
sensitivity_new_test_df = load_ecg_dataframe_1d(
    new_csv_path,
    new_data_dir,
    # imputer_dir=multitask_clinical_model_path,
    do_split=False,
    lvef_threshold=40,
)

In [11]:
import pickle
from scipy.signal import resample
import biosppy
from tqdm.auto import tqdm
tqdm.pandas()

ECG_FORMAT_TO_PAPER_PIXEL_SAMPLE_RATE = {
    'old': 294,
    'new': 280
}

def apply_rpeaks_normalization(lead_array_list, rpeak_index, ecg_format):
    paper_pixel_sample_rate = ECG_FORMAT_TO_PAPER_PIXEL_SAMPLE_RATE[ecg_format]
    start = max(0, int(rpeak_index - 0.5 * paper_pixel_sample_rate))
    end = int(rpeak_index + 1.5 * paper_pixel_sample_rate)
    return [lead[start:end] for lead in lead_array_list]

def has_empty_leads(lead_array_list):
    for lead in lead_array_list:
        if len(lead) == 0:
            return True
    return False

def filter_out_empty_leads(dataframe):
    print(f"Shape Before filter out: {dataframe.shape}")
    dataframe['has_empty_leads'] = dataframe['lead_arrays'].progress_apply(lambda lead_list: has_empty_leads(lead_list))
    dataframe = dataframe[dataframe['has_empty_leads'] != True].reset_index(drop=True)
    print(f"Shape After filter out: {dataframe.shape}")
    return dataframe

def resample_leads(lead_array_list, sample_rate: int = 200):
    for i, lead in enumerate(lead_array_list):
        try:
            lead_array_list[i] = resample(lead, sample_rate)
        except:
            print(f"Error resampling lead {i}")
            print(lead)
            print(len(lead))

    return lead_array_list

def process_dataframe(dataframe, sample_rate: int = 200):
    dataframe['lead_arrays'] = dataframe[constants.path_column_name].progress_apply(lambda x: pickle.load(open(x, "rb")))
    dataframe['has_empty_leads'] = dataframe['lead_arrays'].progress_apply(lambda lead_list: has_empty_leads(lead_list))
    print(f"Shape Before filter out: {dataframe.shape}")
    dataframe = dataframe[dataframe['has_empty_leads'] != True].reset_index(drop=True)
    print(f"Shape After filter out: {dataframe.shape}")
    dataframe['lead_arrays'] = dataframe.progress_apply(lambda row: apply_rpeaks_normalization(row['lead_arrays'], row['median_first_rpeak_index'], row['ecg_format']), axis=1)
    dataframe['has_empty_leads'] = dataframe['lead_arrays'].progress_apply(lambda lead_list: has_empty_leads(lead_list))
    dataframe = dataframe[dataframe['has_empty_leads'] != True].reset_index(drop=True)
    print(f"Shape After filter out: {dataframe.shape}")
    dataframe['lead_arrays'] = dataframe['lead_arrays'].progress_apply(lambda lead_list: resample_leads(lead_list, sample_rate))
    return dataframe

old_test_df = process_dataframe(old_test_df, 200)
new_test_df = process_dataframe(new_test_df, 200)

  0%|          | 0/895 [00:00<?, ?it/s]

  0%|          | 0/895 [00:00<?, ?it/s]

Shape Before filter out: (895, 35)
Shape After filter out: (895, 35)


  0%|          | 0/895 [00:00<?, ?it/s]

  0%|          | 0/895 [00:00<?, ?it/s]

...

Shape Before filter out: (1264, 35)
Shape After filter out: (1264, 35)


  0%|          | 0/1264 [00:00<?, ?it/s]

  0%|          | 0/1264 [00:00<?, ?it/s]

Shape After filter out: (1263, 35)


  0%|          | 0/1263 [00:00<?, ?it/s]

In [7]:
# from mtecg.datasets_1d import ECG1DDataset
# from torch.utils.data import DataLoader

# dataset = ECG1DDataset(
#     dataframe=old_test_df,
#     label_column="scar_cad",
# )
# dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

# dataloader_iter = iter(dataloader)

# for i in range(1):
#     data = next(dataloader_iter)
#     print(data[0].shape)
#     print(data[1].shape)


# leads_batch = data[0]
# batch_size = leads_batch.shape[0]
# leads_batch = leads_batch.view(batch_size * 12, 1, -1)
# leads_batch = leads_batch.unsqueeze(1)
# leads_batch.shape


# model = singletask_scar_classifier.model
# lead_embeddings = model.model(leads_batch.to(device))

# lead_embeddings = lead_embeddings.view(batch_size, 12, -1)
# lead_embeddings.shape
# torch.mean(lead_embeddings, dim=1).shape

In [12]:
# For convenience.

EVAL_DATA_MAP = {
    "old-test": {"data": old_test_df, "save_suffix": "old_test",},
    # "old-test-sensitivity": {"data": sensitivity_old_test_df, "save_suffix": "old_test_sensitivity",},
    "new-test": {"data": new_test_df, "save_suffix": "new_test",},
    # "new-test-sensitivity": {"data": sensitivity_new_test_df, "save_suffix": "new_test_sensitivity",},
    # "control-test": {"data": control_test_df, "save_suffix": "control_test",},
}

TEST_SET_SAVE_SUFFIX_LIST = [param_dict["save_suffix"] for param_dict in EVAL_DATA_MAP.values()]


def evaluate_and_save(
    classifier: ECGClassifier,
    save_dir: str,
    average: str = "weighted",
    prediction_csv_name_pattern: str = "prediction_{save_suffix}.csv",
    metric_csv_name_pattern: str = "metrics_{save_suffix}.csv",
):
    for test_set_name, param_dict in tqdm(EVAL_DATA_MAP.items()):
        dataframe, save_suffix = param_dict["data"], param_dict["save_suffix"]
        if "control" in test_set_name:
            result_dataframe, metric_dataframe = evaluate_from_dataframe_1d(
                dataframe,
                classifier,
                is_control_population=True,
                average=average,
                )
        else:
            result_dataframe, metric_dataframe = evaluate_from_dataframe_1d(dataframe, classifier)

        result_save_path = op.join(save_dir, prediction_csv_name_pattern.format(save_suffix=save_suffix))
        metric_save_path = op.join(save_dir, metric_csv_name_pattern.format(save_suffix=save_suffix))

        result_dataframe.to_csv(result_save_path, index=False)
        metric_dataframe.to_csv(metric_save_path, index=True)

## **Evaluation**

### **Baseline**

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score

def get_baseline_metrics(
    save_dir: str = None,
    label_column_name: str = "scar_cad",
    metric_csv_name_pattern: str = "{save_suffix}.csv",
    average: str = "weighted",
    ):
    baseline_metric_dict = {}
    for test_set_name, param_dict in EVAL_DATA_MAP.items():
        if "control" in test_set_name:
            continue
        dataframe = param_dict["data"]
        baseline_predictions = np.zeros(len(dataframe))

        # get specificity from confusion matrix
        tn, fp, fn, tp = confusion_matrix(dataframe[label_column_name], baseline_predictions).ravel()
        specificity = tn / (tn+fp)
        fpr = fp / (fp+tn)

        baseline_metric_dict[test_set_name] = {
            "Accuracy": accuracy_score(dataframe[label_column_name], baseline_predictions),
            "Sensitivity": None,
            "Specificity": specificity,
            "F1": f1_score(dataframe[label_column_name], baseline_predictions, average=average),
            "AUC": None,
            "FPR": fpr,
            "FNR": None,
        }

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        for test_set_name, param_dict in EVAL_DATA_MAP.items():
            if "control" in test_set_name:
                continue
            save_suffix = param_dict["save_suffix"]
            metric_save_path = op.join(save_dir, metric_csv_name_pattern.format(save_suffix=save_suffix))
            pd.DataFrame(baseline_metric_dict[test_set_name], index=[0]).T.to_csv(metric_save_path, index=True)

    return baseline_metric_dict

In [10]:
scar_baseline_metric_dict = get_baseline_metrics(
    save_dir=op.join("../resources/statistics/1d/scar_baseline_metrics"),
    label_column_name="scar_cad",
)

lvef_baseline_metric_dict = get_baseline_metrics(
    save_dir=op.join("../resources/statistics/1d/lvef_baseline_metrics"),
    label_column_name="lvef",
)

### **single-task**

In [19]:
evaluate_and_save(singletask_scar_classifier, save_dir=singletask_scar_model_path)
# evaluate_and_save(singletask_lvef_classifier, save_dir=singletask_lvef_model_path)

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [24]:
prediction_df = pd.read_csv(op.join(singletask_lvef_model_path, "prediction_old_test.csv"))
prediction_df.lvef_label.value_counts(), prediction_df.lvef_prediction.value_counts()

(0    722
 1    173
 Name: lvef_label, dtype: int64,
 0    611
 1    284
 Name: lvef_prediction, dtype: int64)

In [25]:
prediction_df = pd.read_csv(op.join(singletask_lvef_model_path, "prediction_new_test.csv"))
prediction_df.lvef_label.value_counts(), prediction_df.lvef_prediction.value_counts()

(0    1035
 1     228
 Name: lvef_label, dtype: int64,
 0    1018
 1     245
 Name: lvef_prediction, dtype: int64)

### **single-task-clinical**

In [ ]:
evaluate_and_save(singletask_scar_clinical_classifier, save_dir=singletask_scar_clinical_model_path)
evaluate_and_save(singletask_lvef_clinical_classifier, save_dir=singletask_lvef_clinical_model_path)

### **multi-task**

In [10]:
evaluate_and_save(multitask_classifier, save_dir=multitask_model_path)

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

**multi-task-old-format**

In [28]:
evaluate_and_save(multitask_old_format_classifier, save_dir=multitask_old_format_model_path)

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

**multi-task-transferred**

In [29]:
evaluate_and_save(multitask_transferred_classifier, save_dir=multitask_transferred_model_path)

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

### **multi-task-clinical**

In [ ]:
# Overwrite the EVAL_DATA_MAP to only evaluate the test sets with available clinical data.
EVAL_DATA_MAP = {
    "old-test": {"data": old_test_df, "save_suffix": "old_test",},
    "old-test-sensitivity": {"data": sensitivity_old_test_df, "save_suffix": "old_test_sensitivity",},
    "new-test": {"data": new_test_df, "save_suffix": "new_test",},
    "new-test-sensitivity": {"data": sensitivity_new_test_df, "save_suffix": "new_test_sensitivity",},
}

evaluate_and_save(multitask_clinical_classifier, save_dir=multitask_clinical_model_path)

[scar] Prevalence-specific Evaluation

In [ ]:
import random

def sample(dataframe, num_samples: int = 500, prevalence: float = 7.9):
    """
    Sample a dataframe to a prevalence of 7.9% (the prevalence of scar in the dataset).
    """
    positive_dataframe = dataframe[dataframe.scar_label == 1].reset_index(drop=True).copy()
    negative_dataframe = dataframe[dataframe.scar_label == 0].reset_index(drop=True).copy()
    
    num_positive_samples = int(num_samples * prevalence / 100)
    num_negative_samples = num_samples - num_positive_samples
    
    random_state = random.randint(0, 1000)
    positive_sample_dataframe = positive_dataframe.sample(n=num_positive_samples, random_state = random_state).reset_index(drop=True)
    negative_sample_dataframe = negative_dataframe.sample(n=num_negative_samples, random_state = random_state).reset_index(drop=True)
    sampled_dataframe = pd.concat([positive_sample_dataframe, negative_sample_dataframe]).reset_index(drop=True)
    return sampled_dataframe

In [ ]:
import scipy.stats as st
from mtecg.evaluation import calculate_metrics

num_sample_per_iteration = 500
# num_iteration_list = [20, 50, 200]
num_iteration_list = [1000]


multitask_prediction_df = pd.read_csv(op.join(multitask_model_path, "prediction_new_test.csv"))

for num_iteration in tqdm(num_iteration_list):
    scar_auc_list = []
    scar_f1_list = []
    for i in tqdm(range(num_iteration)):
        sampled_test_df = sample(multitask_prediction_df, num_samples = num_sample_per_iteration)
        metric_df = calculate_metrics(sampled_test_df, tasks=["scar"])
        # _, metric_df = evaluate_from_dataframe(sampled_test_df, multitask_classifier)
        # _, metric_df = evaluate_from_dataframe(sampled_clinical_test_df, multitask_clinical_classifier)
        scar_auc_list.append(metric_df.T["AUC"][0])
        scar_f1_list.append(metric_df.T["F1"][0])

    # Create 95% confidence interval for population mean scar auc.
    auc_confidence_interval_tuple = st.t.interval(
        alpha=0.95,
        df=len(scar_auc_list)-1,
        loc=np.mean(scar_auc_list),
        scale=st.sem(scar_auc_list)
        )
    
    f1_confidence_interval_tuple = st.t.interval(
        alpha=0.95,
        df=len(scar_f1_list)-1,
        loc=np.mean(scar_f1_list),
        scale=st.sem(scar_f1_list)
        )

    auc_summary_df = pd.DataFrame(
        { 
            "mean": [np.mean(scar_auc_list)],
            "std": [np.std(scar_auc_list)],
            "lower_bound_ci": [auc_confidence_interval_tuple[0]],
            "upper_bound_ci": [auc_confidence_interval_tuple[1]],
            }
    )\
        .round(4)

    f1_summary_df = pd.DataFrame(
        { 
            "mean": [np.mean(scar_f1_list)],
            "std": [np.std(scar_f1_list)],
            "lower_bound_ci": [f1_confidence_interval_tuple[0]],
            "upper_bound_ci": [f1_confidence_interval_tuple[1]],
            }
    )\
        .round(4)

    auc_summary_df.to_csv(op.join(multitask_model_path, f"prevalence_specific_auc_{num_sample_per_iteration}_{num_iteration}.csv"), index=False)
    f1_summary_df.to_csv(op.join(multitask_model_path, f"prevalence_specific_f1_{num_sample_per_iteration}_{num_iteration}.csv"), index=False)

## **Save Prediction Probabilities on Each Test Set as a single file**

In [3]:
import pandas as pd
from typing import Dict, List

# A function to read the predictions from the save csv file in each model folder.
# The probability columns of each task are then concatenated into a single dataframe.
# The probability columns are in the format f"{task}_probability".

def get_probabilities(
    model_name_to_dir_map: Dict[str, str],
    test_set_suffix_list: List[str],
    probability_column_name_pattern: str = "{task}_probability",
    prediction_csv_name_pattern: str = "prediction_{test_set_suffix}.csv",
    task: str = "scar",
) -> List[pd.DataFrame]:
    """
    Get the probabilities of the given task from the predictions of the models.
    """
    probability_column_name = probability_column_name_pattern.format(task=task)

    test_set_to_probability_dataframe_dict = {}
    for test_set_suffix in test_set_suffix_list:
        if "control" in test_set_suffix:
            continue
        model_name_to_probabilities_dict = {}
        for model_name, model_dir in model_name_to_dir_map.items():
            filename = prediction_csv_name_pattern.format(test_set_suffix=test_set_suffix)
            prediction_path = op.join(model_dir, filename)
            prediction_dataframe = pd.read_csv(prediction_path)
            if "true_label" not in model_name_to_probabilities_dict.keys():
                model_name_to_probabilities_dict["true_label"] = prediction_dataframe[f"{task}_label"]
            model_name_to_probabilities_dict[model_name] = prediction_dataframe[probability_column_name]
        probability_dataframe = pd.DataFrame(model_name_to_probabilities_dict)
        test_set_to_probability_dataframe_dict[test_set_suffix] = probability_dataframe
    return test_set_to_probability_dataframe_dict

In [7]:
singletask_scar_model_path = "../trained_models/1d/single-task-scar/resnet34d_200"
singletask_lvef_model_path = "../trained_models/1d/single-task-lvef/resnet34d_200_LVEF50"
multitask_model_path = "../trained_models/1d/multi-task/resnet34d_200_LVEF50"
multitask_old_format_model_path = "../trained_models/1d/multi-task-old-format/resnet34d_200_LVEF50"
multitask_transferred_model_path = "../trained_models/1d/multi-task-transferred/resnet34d_200_LVEF50"

probability_save_dir = "../resources/prediction_probabilities_1d"
os.makedirs(probability_save_dir, exist_ok=True)

In [13]:
tasks = ["scar", "lvef"]
for task in tasks:
    model_name_to_dir_map = {
        "multi-task-old-format": multitask_old_format_model_path,
        "multi-task-transferred": multitask_transferred_model_path,
        "single-task": singletask_scar_model_path if task == "scar" else singletask_lvef_model_path,
        "multi-task": multitask_model_path,
    }

    test_set_to_probability_dataframe_dict = get_probabilities(
        model_name_to_dir_map=model_name_to_dir_map,
        test_set_suffix_list=TEST_SET_SAVE_SUFFIX_LIST,
        task=task,
    )

    for test_set_suffix, probability_dataframe in test_set_to_probability_dataframe_dict.items():
        probability_save_path = op.join(probability_save_dir, f"{task}_probabilities_{test_set_suffix}.csv")
        probability_dataframe.to_csv(probability_save_path, index=False)